## Проект "Классификация автомобилей по изображению"

**Цель:** Классифицировать модель автомобиля по фотографии.

**Задание:** Обучить нейронную сеть определять модель автомомбиля по изображению. Модели автомобилей, представленные в тренировочных данных ('Приора', 'Ford Focus', 'Самара', 'ВАЗ-2110', 'Жигули', 'Нива', 'Калина',   'ВАЗ-2109', 'Volkswagen Passat', 'ВАЗ-21099') 

**Данные:** 
1. train.csv - обучающая выборка
2. train.zip - картинки к обучающей выборке
3. sample-submission.csv - тестовая выборка
4. test.zip - картинки к тестовой выборке




In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# проверяем, что видеокарта подключена
!nvidia-smi -L

In [ ]:
# Устанавливаем ImageDataAugmentor:
#  это настраиваемый генератор данных изображения для tensorflow.keras, который поддерживает альбументацию.
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor

In [ ]:
# Устанавливаем последнюю версию
!pip install -q -U albumentations

In [ ]:
!pip install -q efficientnet

## Библиотеки:

In [ ]:
import os #Для работы с файлами
import sys 
import random
import zipfile
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image #Для отрисовки изображений
%matplotlib inline
from tqdm.notebook import tqdm # ход выполнения

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import *
from tensorflow.keras.activations import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.layers.experimental.preprocessing import *
from tensorflow.keras.losses import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.optimizers.schedules import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from ImageDataAugmentor.image_data_augmentor import *
import albumentations as A

In [ ]:
# Версии
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

## Данные

In [ ]:
# Пути
DATA_PATH = '../input/sf-dl-car-classification/'
PATH = "../working/car/" # рабочая директория

In [ ]:
# Распаковываем архивы
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile(DATA_PATH + data_zip,"r") as z:
        z.extractall(PATH)
        z.close()
        
print(os.listdir(PATH))

In [ ]:
# Загружаем датасеты
train_df = pd.read_csv(DATA_PATH+"train.csv")
test_sub = pd.read_csv(DATA_PATH+"sample-submission.csv")

In [ ]:
train_df.sample(3)

In [ ]:
train_df.info()

In [ ]:
sns.countplot(x='Category', data=train_df, color='dodgerblue')

В обучающей выборке пропусков нет. Объекты распределены по классам не совсем равномерно, но в каждой категории достаточно объектов. 

In [ ]:
# Список названий моделей - имена классов
class_names = [
  'Приора', #0
  'Ford Focus', #1
  'Самара', #2
  'ВАЗ-2110', #3
  'Жигули', #4
  'Нива', #5
  'Калина', #6
  'ВАЗ-2109', #7
  'Volkswagen Passat', #8
  'ВАЗ-21099' #9
]

**Посмотрим изображения автомобилей**

In [ ]:
# Пути до изображений
train_img_path = './car/train/'
test_img_path = './car/test_upload/'

In [ ]:
from IPython.core.pylabtools import figsize
fig, ax = plt.subplots(3, 3, figsize=(20,15))
for i, axi in enumerate(ax.flat):
    car_path = train_img_path + str(i)+'/'
    img_path = car_path + random.choice(os.listdir(car_path))
    img = Image.open(img_path)
    axi.imshow(img)
    axi.set_title(class_names[i])
plt.show()

Фотографии в разном разрешении, в основном размером 640 на 480. Автомобили сняты с разного ракурса.

## Подготовка данных для нейросети

**Основные насройки**

In [ ]:
LR = 0.001
IMAGE_SIZE = (160, 200)
BATCH_SIZE = 45
RANDOM_SEED = 42
EPOCHS = 30
VALIDATION_SPLIT=0.25 # процент данных на валидацию
OPTIMIZER=optimizers.Adam(learning_rate=LR)



**Аугментация данных**

In [ ]:
augmentations = A.Compose([
     #A.Blur(p=0.2, blur_limit=(3, 3)),
    A.CLAHE(p=0.4),
    A.CoarseDropout(p=0.3, max_holes=16, max_height=12, max_width=12, min_holes=6, min_height=6, min_width=6),
    A.ElasticTransform(p=0.3),
    A.Equalize( p=0.4, mode='cv', by_channels=True),
    A.GaussNoise(p=0.4, var_limit=(10, 30)),
    A.GridDistortion(p=0.3),
    A.HorizontalFlip(p=0.5),
    A.HueSaturationValue(p=0.2),  
     #A.MultiplicativeNoise(p=0.3, multiplier=(0.8, 1.5)),
    A.OpticalDistortion(p=0.3, distort_limit=(-0.3, 0.3), shift_limit=(-0.05, 0.05), interpolation=3),
    A.RandomBrightnessContrast(p=0.5, brightness_limit=(-0.1, 0.1), contrast_limit=(-0.2, 0.2)),
    A.RandomRain(p=0.2, slant_lower=-6, slant_upper=-2, drop_length=11, drop_width=1, blur_value=1, brightness_coefficient=1), 
    A.Rotate(p=0.4, limit=(-15, 15), border_mode=2),
])

train_datagen = ImageDataAugmentor(
    #rescale = 1./255, # Нормализуем яркости от 0 до 1
    augment = augmentations,
    validation_split=VALIDATION_SPLIT # Указываем разделение изображений на обучающую и тестовую выборку
)

valid_datagen = ImageDataAugmentor(
    #rescale = 1./255, # нормализуем яркости от 0 до 1
    validation_split=VALIDATION_SPLIT # Указываем разделение изображений на обучающую и тестовую выборку
)

test_datagen = ImageDataGenerator(
    #rescale = 1./255 # нормализуем яркости от 0 до 1
)

**Генерация данных**

In [ ]:
train_generator = train_datagen.flow_from_directory(
   train_img_path,
   target_size=IMAGE_SIZE,
   batch_size=BATCH_SIZE,
   class_mode='categorical',
   shuffle=True,
   seed=RANDOM_SEED,
   subset='training'
    
)

valid_generator = valid_datagen.flow_from_directory(
    train_img_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=RANDOM_SEED,
    subset='validation'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_sub,
    directory=test_img_path,
    x_col='Id',
    y_col=None,
    class_mode=None,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False

)

Посмотрим, что получилось:

In [ ]:
def show_first_image(generator, count=6, labels=True, figsize=(20, 5), normalized=False):
    """Функция выводит на экран изображения, обработанные генератором"""

    generator = itertools.islice(generator, count) # Создаёт итератор по ограниченному подмножеству элементов переданного объекта.
    fig, ax = plt.subplots(nrows=1, ncols=count, figsize=figsize)
  
    for batch, ax in zip(generator, ax):
           
        if labels:
            img_batch, labels_batch = batch
            img, label = img_batch[0], np.argmax(labels_batch[0])
        else:
            img_batch = batch
            img = img_batch[0]
    
        if not normalized:
            img = img.astype(np.uint8)
    
        ax.imshow(img)
      # метод imshow принимает одно из двух:
      # - изображение в формате uint8, яркость от 0 до 255
      # - изображение в формате float, яркость от 0 до 1

        if labels:
            ax.set_title(f'Class: {label}')
  
    plt.show()




In [ ]:
print('Обучающая выборка:')
show_first_image(train_generator)

print('Валидационная выборка:')
show_first_image(valid_generator)

print('Тестовая выборка:')
show_first_image(test_generator, labels=False)


## Построение модели и обучение

In [ ]:
# Загружаем предобученную сеть EfficientNetB5:

base_model = tf.keras.applications.EfficientNetB5(weights='imagenet', include_top=False, input_shape=(*IMAGE_SIZE, 3))

In [ ]:
# Если надо посмотреть - раскомментировать
# base_model.summary()

In [ ]:
# Сохранение лучшей модели
checkpoint = ModelCheckpoint('best_model.hdf5', monitor=['val_accuracy'], verbose=1, mode='max')
# Останавливаемся если наша модель уже не обучается
earlystop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, )
# Планирование скорости обучения
#callback_learing_rate = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x),

callbacks_list = [checkpoint, earlystop] #, callback_learing_rate

### 1 Шаг

**Модель**

In [ ]:
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D(),) # Объединяет признаки в единый вектор
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
base_model.trainable = True

# Заморозим веса, чтоб базовая модель не обучалась
first_layers_disable = len(base_model.layers)

for layer in base_model.layers[:first_layers_disable]:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
# Проверяем сколько и каких слоев обучаем:
print(f'Обучаем: {len(model.trainable_variables)} слоя(ев)')

for layer in model.layers:
    print(layer, layer.trainable)

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=['accuracy']
)

**Обучение**

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples//valid_generator.batch_size,
    epochs=EPOCHS,
    callbacks=callbacks_list
)

In [ ]:
# Сохраняем модель
model.load_weights('best_model.hdf5')
model.save('../working/model_step_1')

In [ ]:
scores = model.evaluate_generator(valid_generator, steps=len(valid_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
def history_plot(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
 
    epochs = range(len(acc))
 
    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
 
    plt.figure()
 
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
 
    plt.show()

In [ ]:
history_plot(history)

### 2 Шаг

**Модель**

In [ ]:
# загружаем модель
# model = load_model('../working/model_step_1')

In [ ]:
# Разаморозим половину весов
model.layers[0].trainable = True

# Заморозим первую половину слоев базовой модели
first_layers_disable = len(model.layers[0].layers)//2
print(f'Заморожено первых {first_layers_disable} слоя(ев)')

for layer in model.layers[0].layers[:first_layers_disable]:
    layer.trainable = False

In [ ]:
# Проверяем сколько и каких слоев обучаем:
print(f'Обучаем: {len(model.trainable_variables)} слоя(ев)')

for layer in model.layers:
    print(layer, layer.trainable)

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=['accuracy']
)

In [ ]:
model.summary()

**Обучение**

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples//valid_generator.batch_size,
    epochs=EPOCHS,
    callbacks=callbacks_list
)

In [ ]:
# Сохраняем модель
model.load_weights('best_model.hdf5')
model.save('../working/model_step_2')

In [ ]:
scores = model.evaluate_generator(valid_generator, steps=1, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
history_plot(history)

### 3 Шаг

**Модель**

In [ ]:
# Загружаем модель
# model = load_model('../working/model_step_2')

In [ ]:
#model.load_weights('best_model.hdf5')

In [ ]:
# Разаморозим 3/4 части весов
model.layers[0].trainable = True

# Заморозим 1/4 часть первых слоев базовой модели
first_layers_disable = len(model.layers[0].layers)//4
print(f'Заморожено первых {first_layers_disable} слоя(ев)')

for layer in model.layers[0].layers[:first_layers_disable]:
    layer.trainable = False

In [ ]:
# Проверяем сколько и каких слоев обучаем:
print(f'Обучаем: {len(model.trainable_variables)} слоя(ев)')

for layer in model.layers:
    print(layer, layer.trainable)

In [ ]:
LR = 0.0005 # learning_rate
model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=['accuracy']
)

In [ ]:
model.summary()

**Обучаем**

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples//valid_generator.batch_size,
    epochs=EPOCHS,
    callbacks=callbacks_list
)

In [ ]:
# Сохраняем модель
model.load_weights('best_model.hdf5')
model.save('../working/model_step_3')

In [ ]:
scores = model.evaluate_generator(valid_generator, steps=1, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
history_plot(history)

### 4 Шаг

**Модель**

In [ ]:
# загружаем модель
# model = load_model('../working/model_step_3')

In [ ]:
# Разаморозим все веса
model.layers[0].trainable = True

# Проверяем сколько и каких слоев обучаем:
print(f'Обучаем: {len(model.trainable_variables)} слоя(ев)')

for layer in model.layers:
    print(layer, layer.trainable)

In [ ]:
LR = 0.0001 # learning_rate
model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=['accuracy']
)

**Обучаем**

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples//valid_generator.batch_size,
    epochs=EPOCHS,
    callbacks=callbacks_list
)

In [ ]:
# Сохраняем модель
model.load_weights('best_model.hdf5')
model.save('../working/model_step_4')

In [ ]:
scores = model.evaluate_generator(valid_generator, steps=1, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
history_plot(history)

## Предсказываем на тестовых данных

In [ ]:
test_generator.reset()
predictions = model.predict_generator(test_generator, steps=len(test_generator), verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [ ]:
filenames_with_dir=test_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission.csv', index=False)
print('Save submit')

In [ ]:
submission.head()